In [360]:
import os

# Export env vars to limit number of threads to use
num_threads = "4"
os.environ["OMP_NUM_THREADS"] = num_threads 
os.environ["OPENBLAS_NUM_THREADS"] = num_threads
os.environ["MKL_NUM_THREADS"] = num_threads 
os.environ["VECLIB_MAXIMUM_THREADS"] = num_threads
os.environ["NUMEXPR_NUM_THREADS"] = num_threads

# Only use CPU, hide GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [361]:
from transformers import BertTokenizer, BertForMaskedLM, BertForTokenClassification, BertConfig, BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
#Import SummaryWriter for Tensorboard logging
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import (DataLoader, TensorDataset)
# Load Pytorch Geometric
from torch_geometric.data import Data
import torch_geometric.data as tg_data
import torch_geometric.utils as tg_utils
import torch_geometric.nn as tg_nn
import evaluate
# Evaluation metrics for NER task
from seqeval.metrics import classification_report
# Support for IOBES style NER labels
from seqeval.scheme import IOBES
import numpy as np
# Progress bar
from tqdm import tqdm
# Easy file reading
import glob
import pandas as pd
import importlib
import random
import pickle

In [362]:
import utilities as utils
# Reload library if changed
importlib.reload(utils)

<module 'utilities' from '/home/shrdlu/cdaniel/syntrans/utilities.py'>

In [363]:
# Limit no. of threads used by Pytorch
torch.set_num_threads = int(num_threads)

In [364]:
PID = os.getpid()
PGID = os.getpgid(PID)
print(f"PID: {PID}, PGID: {PGID}")

PID: 263, PGID: 149


In [365]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [366]:
activeMode= "develop"

In [367]:
configuration_csv = pd.read_csv(f"./config/{activeMode}.csv", dtype=str, sep=";")
config = utils.configureParameters(configuration_csv)
print(f"Task: {config.task}")
print(f"Model path: {config.saved_model_path}")
print(f"Data path: {config.data_path}")
print(f"Tokenizer: {config.tokenizer}")
print(f"Batch size: {config.batch_size}")
print(f"Epochs: {config.epochs}")
print(f"Learning rate: {config.learning_rate}")
print(f"Sequence length: {config.sequence_length}")
print(f"Training: {config.train_model}")



Task: ner
Model path: bert-base-uncased
Data path: ./data/ud/UD_English-GUM/
Tokenizer: bert-base-uncased
Batch size: 2
Epochs: 1
Learning rate: 5e-05
Sequence length: 136
Training: False


In [368]:
ner_tags_list = ['O','<unk>', 'B-CARDINAL', 'E-CARDINAL', 'S-PERSON', 'S-CARDINAL', 'S-PRODUCT', 'B-PRODUCT', 'I-PRODUCT', 'E-PRODUCT', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'E-WORK_OF_ART', 'B-PERSON', 'E-PERSON', 'S-GPE', 'B-DATE', 'I-DATE', 'E-DATE', 'S-ORDINAL', 'S-LANGUAGE', 'I-PERSON', 'S-EVENT', 'S-DATE', 'B-QUANTITY', 'E-QUANTITY', 'S-TIME', 'B-TIME', 'I-TIME', 'E-TIME', 'B-GPE', 'E-GPE', 'S-ORG', 'I-GPE', 'S-NORP', 'B-FAC', 'I-FAC', 'E-FAC', 'B-NORP', 'E-NORP', 'S-PERCENT', 'B-ORG', 'E-ORG', 'B-LANGUAGE', 'E-LANGUAGE', 'I-CARDINAL', 'I-ORG', 'S-WORK_OF_ART', 'I-QUANTITY', 'B-MONEY', 'I-MONEY', 'E-MONEY', 'B-LOC', 'E-LOC', 'I-LOC', 'B-PERCENT', 'I-PERCENT', 'E-PERCENT', 'S-LOC', 'S-FAC', 'B-EVENT', 'E-EVENT', 'I-EVENT', 'S-MONEY', 'B-LAW', 'I-LAW', 'E-LAW', 'I-NORP', 'I-LANGUAGE', 'S-LAW', 'S-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'E-ORDINAL', '<START>', '<STOP>', "[CLS]", "[SEP]"]
num_labels = len(ner_tags_list)

In [369]:
class BertForNer(BertForTokenClassification):
    """
    Adapted from Huggingface BertForTokenClassification
    """

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None,valid_ids=None,attention_mask_label=None):
        
        # Calculate new embeddings
        sequence_output = self.bert(input_ids, token_type_ids, attention_mask,head_mask=None)[0]
        batch_size,max_len,feat_dim = sequence_output.shape

        # Initialize valid output
        valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32)
        # Calculate new sequence output: ignore non-valid tokens, e.g. subtokens of words
        for batch_idx in range(batch_size):
            valid_idx = -1
            for token_idx in range(max_len):
                    if valid_ids[batch_idx][token_idx].item() == 1:
                        valid_idx += 1
                        valid_output[batch_idx][valid_idx] = sequence_output[batch_idx][token_idx]
        sequence_output = self.dropout(valid_output)
        logits = self.classifier(sequence_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=0)
            # Only keep active parts of the loss
            if attention_mask_label is not None:
                active_loss = attention_mask_label.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits

In [370]:
class SynGNN(nn.Module):
    """
    SynGNN Pytorch module
    based on Pytorch TransformerEncoderLayer implementing the architecture in paper “Attention Is All You Need”. 
    
    """
    def __init__(self, dim_in, dim_hdn, dim_out, num_heads, dim_feedforward=2048, dropout=0.1, activation="relu"):
        """
        :param dim_in: input dimension
        :param dim_hdn: hidden nodes dimension
        :param dim_out: output dimension
        """
        super(nn.TransformerEncoderLayer, self).__init__()
        # Graph attention sublayer
        self.graph_attn = tg_nn.GATv2Conv(dim_in, dim_hdn , heads=num_heads)
        self.linear1 = nn.Linear(dim_hdn, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, dim_out)
        self.norm0 = tg_nn.LayerNorm(dim_in)
        self.norm1 = nn.LayerNorm(dim_hdn)
        self.norm2 = nn.LayerNorm(dim_hdn)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        
        def __setstate__(self, state):
            if 'activation' not in state:
                state['activation'] = F.relu
            super(nn.TransformerEncoderLayer, self).__setstate__(state)

        def forward(self, x, edge_index, batch):
            r"""Pass the input through the encoder layer.
            Args:
                x: node features
                edge_index: graph edges
                batch: current batch
            """
            # Graph attention sublayer
            x_norm = self.norm0(x, batch)
            src2, att_weight = self.graph_attn(x_norm, edge_index)
            src = src + self.dropout1(src2)
            src = self.norm1(src)

            # Feed-Forward-Network sublayer
            src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
            src = src + self.dropout2(src2)
            src = self.norm2(src)
            return src, att_weight
        
        def _get_activation_fn(activation):
            if activation == "relu":
                return F.relu
            elif activation == "gelu":
                return F.gelu

            raise RuntimeError("activation should be relu/gelu, not {}".format(activation))

In [371]:
class SynBertForNer(BertForTokenClassification):
      def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        # self.syngnn = SynGNN()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

      def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None,valid_ids=None,attention_mask_label=None):
         sequence_output = self.bert(input_ids, token_type_ids, attention_mask,head_mask=None)[0]
         batch_size,max_len,feat_dim = sequence_output.shape
         valid_output = torch.zeros(batch_size,max_len,feat_dim,dtype=torch.float32)
         # Calculate sequence output: ignore non-valid tokens, e.g. subtokens of words
         for batch_idx in range(batch_size):
            valid_idx = -1
            for token_idx in range(max_len):
                     if valid_ids[batch_idx][token_idx].item() == 1:
                        valid_idx += 1
                        valid_output[batch_idx][valid_idx] = sequence_output[batch_idx][token_idx]

         # Pipe Bert embeddings into syntactic GAN

         sequence_output = self.dropout(valid_output)
         logits = self.classifier(sequence_output)

         if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=0)
            # Only keep active parts of the loss
            if attention_mask_label is not None:
                  active_loss = attention_mask_label.view(-1) == 1
                  active_logits = logits.view(-1, self.num_labels)[active_loss]
                  active_labels = labels.view(-1)[active_loss]
                  loss = loss_fct(active_logits, active_labels)
            else:
                  loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
         else:
            return logits

In [372]:
tokenizer = BertTokenizer.from_pretrained(config.tokenizer)
if(config.task == 'mlm'):
    model = BertForMaskedLM.from_pretrained(config.saved_model_path)
if(config.task == 'ner'):
    BERTconfig = BertConfig.from_pretrained(config.saved_model_path, num_labels=num_labels, tokenizer = tokenizer)
    model = BertForNer.from_pretrained(config.saved_model_path, from_tf = False, config = BERTconfig)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNer: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNer from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNer from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForNer were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifie

In [373]:
device =  torch.device('cpu')
# Move model to device
model.to(device)

BertForNer(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [374]:
filepath_train_data = config.data_path + '**/*-train.txt'
filepath_test_data = config.data_path + '**/*-test.txt'
filepath_train_syntrees = config.data_path + '**/*-train.syntree'
filepath_test_syntrees = config.data_path + '**/*-test.syntree'
filepath_train_ner_labels = config.data_path + '**/*-train-orig.ner'
filepath_test_ner_labels = config.data_path + '**/*-test-orig.ner'

In [375]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, valid_ids=None, label_mask=None):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        self.valid_ids = valid_ids
        self.label_mask = label_mask

In [376]:
def createMaskedInputs(inputs):
    """
    creates masked input embeddings and labels from tokenized text

    :param inputs: tokenized text
    :return: masked input embeddings and new column labels 
    """ 
    # Clone input ids (tokens) to create labels
    inputs['labels'] = inputs.input_ids.detach().clone()
    # create random array of floats with equal dimensions to input_ids tensor
    rand = torch.rand(inputs.input_ids.shape)
    # create mask array with 15% masked tokens
    mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
        (inputs.input_ids != 102) * (inputs.input_ids != 0)
    # Select indices of each nonzero (= selected) value as token to be masked
    selection = []

    for i in range(inputs.input_ids.shape[0]):
        selection.append(
            torch.flatten(mask_arr[i].nonzero()).tolist()
        )
    # Mask selected tokens: replace with [MASK] code 103 in tensor
    for i in range(inputs.input_ids.shape[0]):
        inputs.input_ids[i, selection[i]] = 103
    
    return inputs

In [377]:
def createNERInputFeatures(sentence_labels_list, label_list, max_seq_length, tokenizer):
    """Loads a list of sentences into a list of input features for the transformer
    
        :return: list of inpt features objects
    """

    # Map NER labels to indices
    # start with 1: 0 reserved for invalid labels e.g. subtoken labels
    label_map = {label : i for i, label in enumerate(label_list,1)}
    label_map['X'] = 0

    features = []
    for (sentence_idx,sentence_label_pair) in enumerate(sentence_labels_list):
    #     sentence = sentence.split(" ")
    #     labellist = labels[sentence_idx]
        if sentence_idx < 2:
            sentence_label_pair
        # Tokenized text of sentence
        tokens = []
        # Token labels for sentence
        labels = []
        # Lists valid labels as 1 and labels to be ignored as 0 (e.g. for the labels for subword tokens which are not counting as separate labels for each token)
        valid = []
        # Mask for transformer indicating which tokens to ignore
        label_mask = []
        for word_label_pair in sentence_label_pair:
            token = tokenizer.tokenize(word_label_pair[0])
            tokens.extend(token)

            label_word = word_label_pair[1]
            for token_idx in range(len(token)):
                # Append label for first token in word, mark as valid
                if token_idx == 0:
                    labels.append(label_word)
                    valid.append(1)
                    label_mask.append(1)
                # Subword tokens: Mark as not valid
                else:
                    labels.append('X')
                    valid.append(0)
                    label_mask.append(1)
        # Sentence exceeds max sequence length: cut to sequence length
        if len(tokens) >= max_seq_length - 1:
            tokens = tokens[0:(max_seq_length - 2)]
            labels = labels[0:(max_seq_length - 2)]
            valid = valid[0:(max_seq_length - 2)]
            label_mask = label_mask[0:(max_seq_length - 2)]
        # Tokens with BERT [CLS] and [SEP] tokens
        ntokens = []
        # Segment ids for BERT
        segment_ids = []
        # Label embedding ids for BERT
        label_ids = []
        # Start segment
        ntokens.append("[CLS]")
        segment_ids.append(0)
        label_ids.append(label_map["[CLS]"])
        # Mark as valid label
        valid.insert(0,1)
        label_mask.insert(0,1)

        # add sentence tokens
        for i, token in enumerate(tokens):
            ntokens.append(token)
            segment_ids.append(0)
            if len(labels) > i:
                label_ids.append(label_map[labels[i]])
        # End segment
        ntokens.append("[SEP]")
        segment_ids.append(0)
        valid.append(1)
        label_mask.append(1)
        label_ids.append(label_map["[SEP]"])

        # Convert tokens to ids
        input_ids = tokenizer.convert_tokens_to_ids(ntokens)
        input_mask = [1] * len(input_ids)
        
        # Pad sentence to sequence length
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            label_ids.append(0)
            valid.append(1)
            label_mask.append(0)
        # Pad labels to sequence length
        while len(label_ids) < max_seq_length:
            label_ids.append(0)
            label_mask.append(0)
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length
        assert len(valid) == max_seq_length
        assert len(label_mask) == max_seq_length
        

        features.append(
        InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_ids=label_ids,
                        valid_ids=valid,
                        label_mask=label_mask))
    return features


In [342]:
class SyntransDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [343]:
def train_ud_tokenizer(tokenizer, tokenizer_name):
    tokenizer_path = "./tokenizers/" + tokenizer_name 
    special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
    ]
    # 30,522 vocab is BERT's default vocab size, feel free to tweak
    vocab_size = 30_522
    # Load data
    text = []
    for ud_file in glob.iglob(config.data_path + '**/UD_English-Pronouns/en_*.txt', recursive=True):

        ud_file = os.path.abspath(ud_file)
        filename = os.path.basename(ud_file)
        print(filename, flush = True)
        tokenizer.train(files=ud_file, vocab_size=vocab_size, special_tokens=special_tokens)
    # make the directory if not already there
    if not os.path.isdir(tokenizer_path):
        os.mkdir(tokenizer_path)
    # save the tokenizer  
    tokenizer.save_model(tokenizer_path)
    


        



In [378]:
def loadSentencesFromFiles(filepath):
    """
    Load sentences from files.

    :param filepath: path to files (supports glob regex)
    :return: list of sentences
    """ 
    sentences = []
    for ud_file in sorted(glob.iglob(filepath, recursive=True)):

        ud_file = os.path.abspath(ud_file)
        filename = os.path.basename(ud_file)
        print(filename, flush = True)
        with open(ud_file, 'r') as fp:
            sentences.extend(fp.read().split('\n'))
    return sentences
        


In [345]:
def loadNERLabelsFromFiles(filepath):
    """
    Load sentences from files.

    :param filepath: path to files (supports glob regex)
    :return: list of NER labels per sentence
    """ 
    labels = []
    for ud_file in sorted(glob.iglob(filepath, recursive=True)):

        ud_file = os.path.abspath(ud_file)
        filename = os.path.basename(ud_file)
        print(filename, flush = True)
        with open(ud_file, 'r') as fp:
            # Split labels file by sentences
            labels.extend(fp.read().split('\n'))
        # Split sentences by tokens
        labels = [x.split("\t") for x in labels]
        # Remove empty line at end of sentence
        [x.remove('') for x in labels] 
        # Split token and NER tags
        labels = [list(map(lambda x:x.split(" ") ,tag_token)) for tag_token in labels]
    return labels

In [346]:
def loadSyntaxTreesFromFiles(filepath):
    """
    Load binary syntax tree files (*.syntree).

    :param filepath: path to files (supports glob regex)
    :return: list of sentence syntax trees
    """ 
    all_syntrees = []
    for syntree_file in sorted(glob.iglob(filepath, recursive=True)):

        syntree_file = os.path.abspath(syntree_file)
        filename = os.path.basename(syntree_file)
        print(filename, flush = True)
        with open(syntree_file, 'rb') as fp:
            all_syntrees.append(pd.read_pickle(fp))
    return all_syntrees

In [384]:
all_syntrees = loadSyntaxTreesFromFiles(filepath_test_syntrees)
print(all_syntrees[0][5])

en_gum-ud-test.syntree
Data(x=[33, 53], edge_index=[2, 64])


In [347]:
# all_syntrees = loadSyntaxTreesFromFiles(filepath_test_syntrees)
# print(all_syntrees[0][0:2])

# syntree_train_loader = tg_data.DataLoader(all_syntrees[0], batch_size=config.batch_size, shuffle=True)


In [348]:
"""
# Print example of tokenized text
sentences = []
for ud_file in glob.iglob(config.data_path + '**/UD_English-Atiien_*.txt', recursive=True):

    ud_file = os.path.abspath(ud_file)
    filename = os.path.basename(ud_file)
    print(filename, flush = True)
    with open(ud_file, 'r') as fp:
        sentences.extend(fp.read().split('\n'))
count = 0
for sentence in sentences:
    # Tokenize data
    inputs = tokenizer(sentence, return_tensors='pt', max_length=config.sequence_length, truncation=True, padding='max_length')
    inputs = createMaskedInputs(inputs)

    # Create dataset from tokenized data
    dataset = SyntransDataset(inputs)
    loader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, shuffle=True)
    if(count==1):
        print(inputs['input_ids'])
        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        print(tokens)
        print(inputs['labels'])
        break
    count=count+1"""

"\n# Print example of tokenized text\nsentences = []\nfor ud_file in glob.iglob(config.data_path + '**/UD_English-Atiien_*.txt', recursive=True):\n\n    ud_file = os.path.abspath(ud_file)\n    filename = os.path.basename(ud_file)\n    print(filename, flush = True)\n    with open(ud_file, 'r') as fp:\n        sentences.extend(fp.read().split('\n'))\ncount = 0\nfor sentence in sentences:\n    # Tokenize data\n    inputs = tokenizer(sentence, return_tensors='pt', max_length=config.sequence_length, truncation=True, padding='max_length')\n    inputs = createMaskedInputs(inputs)\n\n    # Create dataset from tokenized data\n    dataset = SyntransDataset(inputs)\n    loader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, shuffle=True)\n    if(count==1):\n        print(inputs['input_ids'])\n        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])\n        print(tokens)\n        print(inputs['labels'])\n        break\n    count=count+1"

In [349]:
def getMaxSequenceLength(sentences, cutoff_limit_percent=0.9999):
    """
    Calculate maximum sequence length for given data.
    param sentences: list of sentences
    param cutoff_limit_percent: percentage of all samples to accommodate with the max sequence length.
    returns: max sequence length which encompasses cutoff_limit_percent of all data samples
    """
    # Get number of tokens per sentence        
    max_sentence_tokens = 0
    sentence_tokens = {}
    print(f"Amount of samples: {len(sentences)}")
    # Tokenize data
    for sentence in sentences:

        inputs = tokenizer(sentence, return_tensors='pt')
        
        token_count = inputs.input_ids.size(dim=1)
        sentence_tokens[inputs.input_ids.size(dim=1)] = sentence_tokens.get(token_count,0) + 1
        if(token_count > max_sentence_tokens): 
            max_sentence_tokens = token_count
            
    no_tokens = 0
    # Calulate number of samples which should have a sequence length smaller than max_sequence_length
    cutoff = cutoff_limit_percent * len(sentences)
    max_sequence_length = 0
    print(max_sentence_tokens)
    for i in sorted(sentence_tokens):
        # print((i, sentence_tokens[i]), end=" ")
        if(no_tokens <= cutoff):
            no_tokens = no_tokens + sentence_tokens[i]
            max_sequence_length = i

    print(f"Max sequence length: {max_sequence_length} with {cutoff_limit_percent}% of samples smaller")
    return max_sequence_length

In [350]:
#sentences = loadSentencesFromFiles(config.data_path + '**/en_*.txt')

In [351]:
#print(getMaxSequenceLength(sentences))

In [352]:
if (config.train_model):
    print("Training model", flush=True)

    # Load data
    sentences = loadSentencesFromFiles(filepath_train_data)
    # Load NER labels
    print(filepath_train_ner_labels)
    if(config.task == 'ner'):
        sentence_labels_list = loadNERLabelsFromFiles(filepath_train_ner_labels)
        #print(labels[0][0])
        #print(sentences[0][0])
        train_features = createNERInputFeatures(sentence_labels_list, ner_tags_list, config.sequence_length, tokenizer)
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_ids for f in train_features], dtype=torch.long)
        all_valid_ids = torch.tensor([f.valid_ids for f in train_features], dtype=torch.long)
        all_lmask_ids = torch.tensor([f.label_mask for f in train_features], dtype=torch.long)
        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids,all_valid_ids,all_lmask_ids)

        # Print control example of InputFeatures
        print(str(train_features[1].input_ids))
        print(str(train_features[1].input_mask))
        print(str(train_features[1].label_ids))
        print(str(train_features[1].valid_ids))
        print(str(train_features[1].label_mask))
        print(str(train_features[1].segment_ids))


    if(config.task == 'mlm'):
        # Tokenize data
        inputs = tokenizer(sentences, return_tensors='pt', max_length=config.sequence_length, truncation=True, padding='max_length')
        inputs = createMaskedInputs(inputs)

        # Create dataset from tokenized data
        train_data = SyntransDataset(inputs)
    
    loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
    
    # activate training mode
    model.train()

    from torch.optim import AdamW
    # initialize optimizer
    optim = AdamW(model.parameters(), lr=config.learning_rate)

    epochs = config.epochs
    # Tensorboard logging
    log_dir = f"./runs/{config.task}/{config.tokenizer}_E{epochs}_batches{config.batch_size}_LR{config.learning_rate}_SL{config.sequence_length}/"
    writer = SummaryWriter(log_dir=log_dir)
    #writer.add_scalar("LR", config.learning_rate)
    #writer.add_scalar("Batchsize", config.batch_size)

    for epoch in range(epochs):
        scalar_loss = 0
        # setup loop with TQDM and dataloader
        loop = tqdm(loader, leave=True, mininterval=40,maxinterval=120)
        for batch in loop:
            # initialize calculated gradients (from prev step)
            optim.zero_grad()
            # pull all tensor batches required for training
            if (config.task == 'ner'):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids, valid_ids,l_mask = batch
                loss = model(input_ids, segment_ids, input_mask, label_ids,valid_ids,l_mask)
            if (config.task == 'mlm'):
                # pull all tensor batches required for training
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                # process
                outputs = model(input_ids, attention_mask=attention_mask,
                                labels=labels)
                # extract loss
                loss = outputs.loss
            # calculate loss for every parameter that needs grad update
            loss.backward()
            # update parameters
            optim.step()
            # print relevant info to progress bar
            loop.set_description(f'Epoch {epoch}')
            scalar_loss = loss.item()
            loop.set_postfix(loss=scalar_loss)
        # Print info to Tensorboard
        writer.add_scalar("Loss/train", scalar_loss, epoch)
        # Save model after each epoch
        #model.save_pretrained(save_directory=f"./trained_models/mlm/{config.tokenizer}_E{epoch}_batches{config.batch_size}_LR{config.learning_rate}_SL{config.sequence_length}/")
#writer.close()

# Model evaluation

In [353]:
print("Model evaluation\n", flush = True)
# Load test data
if (config.task == 'mlm'):
    # Read test files
    sentences = loadSentencesFromFiles(config.data_path + '**/*-test.txt')
    
    test_inputs = tokenizer(sentences, return_tensors='pt', max_length=config.sequence_length, truncation=True, padding='max_length')
    test_inputs = createMaskedInputs(test_inputs)
    test_data = SyntransDataset(test_inputs)

Model evaluation



In [354]:
if (config.task == 'ner'):
    sentence_labels_list = loadNERLabelsFromFiles(filepath_test_ner_labels)
    test_features = createNERInputFeatures(sentence_labels_list, ner_tags_list, config.sequence_length, tokenizer)
    
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_ids for f in test_features], dtype=torch.long)
    all_valid_ids = torch.tensor([f.valid_ids for f in test_features], dtype=torch.long)
    all_lmask_ids = torch.tensor([f.label_mask for f in test_features], dtype=torch.long)

    # Print control example of InputFeatures
    print(f"Input Ids: \n{str(test_features[5].input_ids)}")
    tokens = []
    for input_id in test_features[5].input_ids:
        tokens.append(tokenizer.convert_ids_to_tokens(input_id))
    print(f"Input Tokens: \n{str(tokens)}")
    print(f"Input Mask: \n{str(test_features[5].input_mask)}")
    print(f"Label Ids: \n{str(test_features[5].label_ids)}")
    print(f"Valid Ids: \n{str(test_features[5].valid_ids)}")
    print(f"Label Mask: \n{str(test_features[5].label_mask)}")
    print(f"Segment Ids: \n{str(test_features[5].segment_ids)}")

    test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids,all_valid_ids,all_lmask_ids)

en_gum-ud-test-orig.ner
Input Ids: 
[101, 25428, 2031, 8920, 1996, 5177, 6194, 2008, 2104, 8091, 9715, 1998, 9787, 13827, 2229, 1031, 1020, 1033, 1025, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Input Tokens: 
['[CLS]', 'psychologists', 'have', 'examined', 'the', 'mental', 'processes', 'that', 'under', '##pin', 'conscious', 'and', 'unconscious', 'bias', '##es', '[', '6', ']', ';', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [356]:

dataloader = torch.utils.data.DataLoader(test_data, batch_size=config.batch_size, shuffle=False)

In [357]:
model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []
label_map = {i : label for i, label in enumerate(ner_tags_list,1)}

references_all = []
predictions_all = []
references_roc_all = []
predictions_roc_all = []

if(config.task == 'mlm'):
    # Setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True, mininterval=20,maxinterval=120)
    for batch in loop:
   
        # Pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).tolist()

        softmax = nn.Softmax(dim = -1)
        with torch.no_grad():
            predictions = model(input_ids)
        predictions = predictions['logits']
        predictions_sm = softmax(predictions)

        # Change type to double to prevent floating point rounding errors
        predictions = predictions.type(torch.float64)
        predictions_sm = softmax(predictions)

        # Get index of argmax
        #y = np.argmax(predictions_sm, axis = -1)
        # y = y.tolist()
        y = torch.topk(predictions, k=1, dim = 2)[1].squeeze()
        y = y.tolist()

    #        words_pred = []
    #        words_true = []
    #        for w in y:
    #            words_pred.append( tokenizer.convert_ids_to_tokens(w))
        
    #        for w in labels[0]:
    #            words_true.append( tokenizer.convert_ids_to_tokens(w))
        
        #print(words_pred[0:50])
        #print(words_true[0:50])
            

        recall_metric = evaluate.load('recall')
        precision_metric = evaluate.load('precision')
        f1_metric = evaluate.load('f1')
        roc_auc_metric = evaluate.load("roc_auc", "multiclass")


        # Go through all samples in batch and add to computation batch
        for idx, pred_batch in enumerate(y):
            references_all.extend(labels[idx])
            predictions_all.extend(pred_batch)
            #precision_metric.add_batch(references=labels[idx], predictions=pred_batch)
            #recall_metric.add_batch(references=labels[idx], predictions=pred_batch)
            #f1_metric.add_batch(references=labels[idx], predictions=pred_batch)
        
        # Calculate ROC
        for batch_idx, pred_batch in enumerate(predictions_sm):
            predictions_roc_all.extend(pred_batch.tolist())
            references_roc_all.extend(labels[batch_idx])
            #roc_auc_metric.add_batch(references=labels[batch_idx], prediction_scores = pred_batch.tolist())
            break
        break

    numberOfBatches = len(loop)
    # List all possible labels
    labels = np.arange(tokenizer.vocab_size)
    with open(f"./logs/Results_{config.task}_{config.tokenizer}_E{config.epochs}_batches{config.batch_size}_LR{config.learning_rate}_SL{config.sequence_length}.txt", "w") as output:
        print(f"Results: {config.tokenizer}, Train={config.train_model} {config.tokenizer}_E{config.epochs}_batches{config.batch_size}_LR{config.learning_rate}_SL{config.sequence_length}", file = output)
        output.write("macro averaging\n")
        output.write(str(recall_metric.compute(references = references_all, predictions = predictions_all, average = 'macro')))
        output.write("\n")
        output.write(str(precision_metric.compute(references = references_all, predictions = predictions_all, average = 'macro', zero_division = 0)))
        output.write("\n")
        output.write(str(f1_metric.compute( references = references_all, predictions = predictions_all, average = 'macro')))
        output.write("\n")
        output.write(str(roc_auc_metric.compute( references = references_roc_all, prediction_scores = predictions_roc_all, average = 'macro', multi_class = 'ovo', labels = labels, max_fpr = 1.0)))
        output.write("\n")
        output.write("weighted averaging\n")
        output.write(str(recall_metric.compute( references = references_all, predictions = predictions_all, average = 'weighted')))
        output.write("\n")
        output.write(str(precision_metric.compute( references = references_all, predictions = predictions_all, average = 'weighted', zero_division = 0)))
        output.write("\n")
        output.write(str(f1_metric.compute( references = references_all, predictions = predictions_all, average = 'weighted')))
        output.write("\n")
        output.write(str(roc_auc_metric.compute( references = references_roc_all, prediction_scores = predictions_roc_all, average = 'weighted', multi_class = 'ovo', labels = labels, max_fpr = 1.0)))
        output.close()





In [359]:
if (config.task == 'ner'):

    sep_token_id = int(ner_tags_list.index("[SEP]")+1)
    cls_token_id = int(ner_tags_list.index("[CLS]")+1)
    unk_token_id = int(ner_tags_list.index("<unk>")+1)
    O_token_id = int(ner_tags_list.index("O")+1)

    special_token_predictions = 0

    for input_ids, input_mask, segment_ids, label_ids,valid_ids,l_mask in tqdm(dataloader, desc="Evaluating"):
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            valid_ids = valid_ids.to(device)
            label_ids = label_ids.to(device)
            l_mask = l_mask.to(device)

            with torch.no_grad():
                logits = model(input_ids, segment_ids, input_mask,valid_ids=valid_ids,attention_mask_label=l_mask)

            softmax = nn.Softmax(dim=2)
            logits = torch.argmax(softmax(logits),dim=2)
            logits = logits.detach().cpu().numpy()
            label_ids = label_ids.to('cpu').numpy()
            input_mask = input_mask.to('cpu').numpy()

            #print(f"First label in labels: {label_ids[1][0]}")

            for label_list_idx, true_sentence_labels in enumerate(label_ids):
                y_true_temp = []
                y_pred_temp = []

                for label_idx, label_id in enumerate(true_sentence_labels):

                    # Skip 0 label
                    if label_id == 0:
                        continue

                    # Skip [CLS] label at sequence beginning
                    if label_id == cls_token_id:
                        continue

                    # Detect [SEP] label at sentence end and ignore [SEP] and sequence padding
                    elif label_id == sep_token_id:
                         y_true.append(y_true_temp)
                         y_pred.append(y_pred_temp)
                         break
                    else:
                        if (logits[label_list_idx][label_idx] == 0):
                            special_token_predictions = special_token_predictions +1
                            logits[label_list_idx][label_idx] = O_token_id
                           
                        y_true_temp.append(label_map[label_id])
                        y_pred_temp.append(label_map[logits[label_list_idx][label_idx]])

    report = classification_report(y_true, y_pred, digits=4)
    with open(f"./logs/{config.task}/Results_{config.task}_{config.tokenizer}_E{config.epochs}_batches{config.batch_size}_LR{config.learning_rate}_SL{config.sequence_length}.txt", "w") as output:
        print("***** Eval results *****")
        print(f"{report}\n Special token predictions: {special_token_predictions}")
        output.write(report)

Evaluating: 100%|██████████| 448/448 [12:57<00:00,  1.73s/it]
/home/shrdlu/cdaniel/venv_syntrans/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [CLS] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shrdlu/cdaniel/venv_syntrans/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <unk> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shrdlu/cdaniel/venv_syntrans/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <STOP> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shrdlu/cdaniel/venv_syntrans/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <START> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shrdlu/cdaniel/venv_syntrans/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetri

***** Eval results *****
              precision    recall  f1-score   support

    CARDINAL     0.0075    0.0921    0.0138       152
        CLS]     0.0000    0.0000    0.0000         0
        DATE     0.0085    0.0556    0.0147       180
       EVENT     0.0000    0.0000    0.0000        14
         FAC     0.0000    0.0000    0.0000        48
         GPE     0.0036    0.0102    0.0054       196
    LANGUAGE     0.0023    0.0200    0.0041        50
         LAW     0.0000    0.0000    0.0000         8
         LOC     0.0000    0.0000    0.0000        45
       MONEY     0.0000    0.0000    0.0000         2
        NORP     0.0035    0.1017    0.0067        59
     ORDINAL     0.0019    0.0208    0.0035        48
         ORG     0.0041    0.0267    0.0071       150
     PERCENT     0.0000    0.0000    0.0000         6
      PERSON     0.0000    0.0000    0.0000       154
     PRODUCT     0.0000    0.0000    0.0000        32
    QUANTITY     0.0000    0.0000    0.0000        16
  

In [ ]:
print("Finished evaluation")

In [ ]:
#confusion_matrix = metrics.confusion_matrix(references_all, predictions_all, labels=labels)
#print(confusion_matrix)
#disp = metrics.ConfusionMatrixDisplay(references_all, predictions_all, labels=labels)
#disp.plot()

In [ ]:
"""
# precision recall curve
from sklearn.metrics import precision_recall_curve, roc_curve
import matplotlib.pyplot as plt
#%matplotlib inline
precision = dict()
recall = dict()
for i in labels:
    precision[i], recall[i], _ = precision_recall_curve(references_roc_all[i],
                                                        predictions_roc_all[i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))
    break
    
plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.show()"""